In [2]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np



In [5]:
#read in files
realisation1 = 'r1i1p1f1'
model1 = 'cesm2'
realisation2 = 'r1i1p1f2'
model2 = 'ukesm'
lut1 = xr.open_dataset('CMIP/tasLut_'+ model1 + '_' + realisation1 + '_obs.nc')
lut2 = xr.open_dataset('CMIP/tasLut_'+ model2 + '_' + realisation2 + '_obs.nc')

lst1 = xr.open_dataset('lstmip/tslsilut_'+ model1 + '_obs.nc')
lst1 = lst1.sel(time = slice(lst1.time[23],lst1.time[166]))

luh = xr.open_dataset('LAF/LAF_obs_new.nc')
luh_dom = xr.open_dataset('LAF/LAF_remap_new.nc')


In [4]:
#make yearly luh data monthly
orig_time = luh.time.values
add_extra = np.append(orig_time,np.datetime64('2015-01-01T00:00:00.000000000'))
full_time = xr.Dataset({"foo": ("time", np.arange(13)), "time": add_extra})

luh2 = luh.reindex(time=full_time.time, fill_value=-3000.0).sortby("time")

luhmon = luh2.resample(time="1M").pad()
luh_new = luhmon.sel(time = slice(luhmon.time[0],luhmon.time[-2]))

luh2_dom = luh_dom.reindex(time=full_time.time, fill_value=-3000.0).sortby("time")
luhmon_dom = luh2_dom.resample(time="1M").pad()
luh_newdom = luhmon_dom.sel(time = slice(luhmon_dom.time[0],luhmon_dom.time[-2]))

In [5]:
#apply mapping to mod1
luh_new=luh_new.assign_coords(time=lut1.time)
luh_new=luh_new.assign_coords(lat=lut1.lat)
luh_new=luh_new.assign_coords(lon=lut1.lon)
mpd1 = lut1.copy(deep = True)
mpd1['prim'] = lut1.tasLut.isel(landuse=0)*luh_new.prim.values
mpd1['crop'] = lut1.tasLut.isel(landuse=2)*luh_new.crop.values
mpd1['urban'] = lut1.tasLut.isel(landuse=3)*luh_new.urban.values



In [6]:
mpd1['urban'] = xr.where((~np.isnan(mpd1.prim)) & (np.isnan(mpd1.urban)),0,mpd1.urban)
mpd1['crop'] = xr.where((~np.isnan(mpd1.prim)) & (np.isnan(mpd1.crop)),0,mpd1.crop)
mpd1['tas'] = mpd1.prim+mpd1.crop+mpd1.urban

In [7]:
mpd1 = mpd1.drop(['prim','crop','urban'])
mpd1.to_netcdf('review_files/mapped_tas_' + model1 + '_new.nc')

In [8]:
#apply mapping to mod2
lut2=lut2.assign_coords(time=luh_new.time)
mpd2 = luh_new.copy(deep = True)
mpd2['prim'] = lut2.tasLut.isel(landuse=0)*luh_new.prim
mpd2['crop'] = lut2.tasLut.isel(landuse=2)*luh_new.crop
mpd2['urban'] = lut2.tasLut.isel(landuse=3)*luh_new.urban
mpd2['tas'] = mpd2.prim+mpd2.crop+mpd2.urban

In [9]:
mpd2['urban'] = xr.where((~np.isnan(mpd2.prim)) & (np.isnan(mpd2.urban)),0,mpd2.urban)
mpd2['crop'] = xr.where((~np.isnan(mpd2.prim)) & (np.isnan(mpd2.crop)),0,mpd2.crop)
mpd2['tas'] = mpd2.prim+mpd2.crop+mpd2.urban

In [11]:
mpd2 = mpd2.drop(['prim','crop','urban'])
mpd2.to_netcdf('review_files/mapped_tas_' + model2 + '_new.nc')

In [3]:
#apply mapping to lst
luh3 = xr.open_dataset('LAF/laf_lst.nc')
luh3 = luh3.sel(time = slice(luh3.time[2],luh3.time[13]))
orig_time = luh3.time.values
add_extra = np.append(orig_time,np.datetime64('2015-01-01T00:00:00.000000000'))
full_time = xr.Dataset({"foo": ("time", np.arange(13)), "time": add_extra})

luh4 = luh3.reindex(time=full_time.time, fill_value=-3000.0).sortby("time")

luhmon1 = luh4.resample(time="1M").pad()
luh_new1 = luhmon1.sel(time = slice(luhmon1.time[0],luhmon1.time[-2]))


a = xr.where((luh_new1.prim >luh_new1.crop)& (luh_new1.prim >luh_new1.urban),1,0)
b = xr.where((luh_new1.crop >luh_new1.prim)& (luh_new1.crop >luh_new1.urban),2,0)
c= xr.where((luh_new1.urban >luh_new1.crop)& (luh_new1.urban > luh_new1.prim ),3,0)

luhdom = a+b+c
luhdom = luhdom.where(luhdom>0)

In [6]:
#apply mapping to lst
luh_new1=luh_new1.assign_coords(time=lst1.time)
luh_new1=luh_new1.assign_coords(lat=lst1.lat)
luh_new1=luh_new1.assign_coords(lon=lst1.lon)
mpd_lst = luh_new1.copy(deep = True)
mpd_lst['prim'] = lst1.tslsiLut.isel(landuse=0)*luh_new1.prim
mpd_lst['crop'] = lst1.tslsiLut.isel(landuse=2)*luh_new1.crop
mpd_lst['urban'] = lst1.tslsiLut.isel(landuse=3)*luh_new1.urban


In [7]:
mpd_lst['urban'] = xr.where((~np.isnan(mpd_lst.prim)) & (np.isnan(mpd_lst.urban)),0,mpd_lst.urban)
mpd_lst['crop'] = xr.where((~np.isnan(mpd_lst.prim)) & (np.isnan(mpd_lst.crop)),0,mpd_lst.crop)
mpd_lst['tas'] = mpd_lst.prim+mpd_lst.crop+mpd_lst.urban


In [9]:
#retain the dominant tile

mpd_lst = mpd_lst.drop(['prim','crop','urban'])
mpd_lst.to_netcdf('review_files/mapped_tslsi_' + model1 + '_new.nc')

In [2]:
import xarray as xr
mpd = xr.open_dataset('review_files/mapped_tas_cesm2.nc')

In [11]:
luh_tas = mpd.dom.median(dim = 'time')
luh_tas = luh_tas.to_dataset()
luh_tas.to_netcdf('review_files/domlaf_air.nc')